In [1]:
import numpy as np
import os


In [16]:
# Load data
with np.load(os.path.join('data', 'data.npz')) as npz_file:
    print(npz_file)
    pass
  
dict=np.load(os.path.join('data', 'data.npz'))

print(dict.files)
#print('X:', X.shape, X.dtype)

#print('y:', y.shape, y.dtype)

['arr_0', 'arr_1']
